# Shallow water equation in 1D

In [1]:
# constants
G = 9.81
H = 4e3
using SparseArrays

In [2]:
function leap_frog_swe(dx::T, dt::T, time_steps::Integer, grid_points::Integer, h0::Function, gamma::T, bc::Symbol = :periodic, g::T=9.81, H::T=4e3) where{T<:AbstractFloat} # better name: leapfrog_ra_advection
    u = zeros(T, grid_points, time_steps) # storage for numerical approximation
    h = zeros(T, grid_points, time_steps)

    # compute initial conditions for h
    cell_boundaries = collect(0:grid_points) .* dx
    cell_centers = cell_boundaries[1:end-1] + 0.5 * diff(cell_boundaries)
    h[:, 1] = h0.(cell_centers)
  
    
    
    # euler step
    upper_diagonal = - ones(grid_points-1)
    lower_diagonal = ones(grid_points-1)
    
    I = [collect(2:grid_points); collect(1:grid_points-1)]  # row indices
    J = [collect(1:grid_points-1); collect(2:grid_points)]  # col indicest
    entries = [lower_diagonal; upper_diagonal]
    
    if bc == :periodic # insert periodic boundary conditions
        I = [I; grid_points; 1]
        J = [J; 1; grid_points]
        entries = [entries; 1.0; -1.0]
    end
    difference_operator = sparse(I, J, entries)

    # perform euler step
    u[:, 2] = (- 0.5 * g * dt / dx) * difference_operator * h[:, 1] + u[:, 1]
    h[:, 2] = (- 0.5 * H * dt / dx) * difference_operator * u[:, 1] + h[:, 1]

    # leap frog
    for k in 2:time_steps-1
        u[:, k+1] = (- g * dt / dx) * difference_operator * h[:, k] + u[:, k-1]
        h[:, k+1] = (- H * dt / dx) * difference_operator * u[:, k] + h[:, k-1]

        # apply the RA filter
        u[:, k] = u[:, k] + gamma * (u[:, k-1] - 2 * u[:, k] + u[:, k+1])
        h[:, k] = h[:, k] + gamma * (h[:, k-1] - 2 * h[:, k] + h[:, k+1])
    end
    return u, h
end

leap_frog_swe (generic function with 4 methods)

In [3]:
h0(x)= (450.0 <= x / 1000.0 <= 550.0) ? 1 : 0
grid_points = 100
time_steps = 1200
dx = 1e5
dt = 10.0
cell_boundaries = dx * collect(0:grid_points)
cell_centers = cell_boundaries[1:end-1] + 0.5 * diff(cell_boundaries)
@info h0.(cell_centers)
@info cell_centers
u, h = leap_frog_swe(dx, dt, time_steps, grid_points, h0, 0.1)

[ Info: [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Info: [50000.0, 150000.0, 250000.0, 350000.0, 450000.0, 550000.0, 650000.0, 750000.0, 850000.0, 950000.0, 1.05e6, 1.15e6, 1.25e6, 1.35e6, 1.45e6, 1.55e6, 1.65e6, 1.75e6, 1.85e6, 1.95e6, 2.05e6, 2.15e6, 2.25e6, 2.35e6, 2.45e6, 2.55e6, 2.65e6, 2.75e6, 2.85e6, 2.95e6, 3.05e6, 3.15e6, 3.25e6, 3.35e6, 3.45e6, 3.55e6, 3.65e6, 3.75e6, 3.85e6, 3.95e6, 4.05e6, 4.15e6, 4.25e6, 4.35e6, 4.45e6, 4.55e6, 4.65e6, 4.75e6, 4.85e6, 4.95e6, 5.05e6, 5.15e6, 5.25e6, 5.35e6, 5.45e6, 5.55e6, 5.65e6, 5.75e6, 5.85e6, 5.95e6, 6.05e6, 6.15e6, 6.25e6, 6.35e6, 6.45e6, 6.55e6, 6.65e6, 6.75e6, 6.85e6, 6.95e6, 7.05e6, 7.15e6, 7.25e6, 7.35e6, 7.45e6, 7.55e6, 7.65e6, 7.75e6, 7.85e6, 7.95e6, 8.05e6, 8.15e6, 8.25e6,

([0.0 0.0 … -0.007966122363259345 -0.007975581763897684; 0.0 0.0 … 0.007211280024655919 0.007272087607646664; … ; 0.0 0.0 … -0.007401896203285408 -0.00733500664036637; 0.0 0.0 … 0.007966122363259345 0.007975581763897684], [0.0 0.0 … -0.044881094274556305 -0.04183900707202571; 0.0 0.0 … -0.044881094274556305 -0.04183900707202571; … ; 0.0 0.0 … 0.02288069697530437 0.025949540453702714; 0.0 0.0 … 0.02288069697530437 0.025949540453702714])

In [10]:
using Plots
using Dates

In [11]:
function create_gif_1dswe(cell_centers::Vector{T}, y::Matrix{T};  filename::String, img_path::String, time_steps::Integer, m::Integer, fps::Integer) where {T<:AbstractFloat}

anim = @animate for k in 1:m:time_steps
        # background: heatmap of h at time t
        # note: transpose h[:,:,t] so axes align intuitively (x horizontal, y vertical)
        plt = plot(cell_centers, y[:, k],
                    ylim = (minimum(y), maximum(y)),
            aspect_ratio = :auto,
            xlabel = "x",
            ylabel = "h(x,t)",
            title = "Numerical Solution of the SWE in d=1 at t = $(k)")

        plt
    end

    gif(anim, filename; fps = fps)
    @info "$(now()) - Saved animation of 1D Shallow Water Equation to $(filename)"
end

function create_plots_1dswe(cell_centers::Vector{T}, y::Matrix{T}, indices::Vector{<:Integer}; filename::String, img_path::String, quantity::String, ylabel::String) where {T<:AbstractFloat}
    plots = []
    ymin = minimum(y)
    ymax = maximum(y)
    for idx in indices
        save_path = "$(img_path)/$(filename)_$(idx).png"
        title = "$(quantity) at time step $(idx)"
        plt = plot(cell_centers, y[:, idx], ylim=(ymin, ymax), title=title)
        savefig(plt, save_path)
    end
end

create_plots_1dswe (generic function with 1 method)

## with staggered grids

In [5]:
function leap_frog_swe_staggered(dx::T, dt::T, time_steps::Integer, grid_points::Integer, h0::Function, gamma::T, bc::Symbol = :periodic, g::T=9.81, H::T=4e3) where{T<:AbstractFloat} # better name: leapfrog_ra_advection
    u = zeros(T, grid_points, time_steps) # storage for numerical approximation
    h = zeros(T, grid_points, time_steps)

    # compute initial conditions for h
    cell_boundaries = collect(0:grid_points) .* dx # location of u
    cell_centers = cell_boundaries[1:end-1] + 0.5 * diff(cell_boundaries) # location of h
    h[:, 1] = h0.(cell_centers)
  
    difference_operator_u = spdiagm( 0 => ones(grid_points), 1 => - ones(grid_points-1))

    # apply boundary conditions
    if bc == :periodic
        difference_operator_u[grid_points, 1] = -1
    end
    
    difference_operator_h = - transpose(difference_operator_u)
    
    # perform euler step
    u[:, 2] = (- g * dt / dx) * difference_operator_h * h[:, 1] + u[:, 1]
    h[:, 2] = (- H * dt / dx) * difference_operator_u * u[:, 1] + h[:, 1]

    # leap frog
    for k in 2:time_steps-1
        u[:, k+1] = (- 2 * g * dt / dx) * difference_operator_h * h[:, k] + u[:, k-1]
        h[:, k+1] = (- 2 * H * dt / dx) * difference_operator_u * u[:, k] + h[:, k-1]

        # apply the RA filter
        u[:, k] = u[:, k] + gamma * (u[:, k-1] - 2 * u[:, k] + u[:, k+1])
        h[:, k] = h[:, k] + gamma * (h[:, k-1] - 2 * h[:, k] + h[:, k+1])
    end
    return u, h
end

leap_frog_swe_staggered (generic function with 4 methods)

In [6]:
h0(x)= (450.0 <= x / 1000.0 <= 550.0) ? 1 : 0
grid_points = 100
time_steps = 1200
dx = 1e5
dt = 10.0
cell_boundaries = dx * collect(0:grid_points)
cell_centers = cell_boundaries[1:end-1] + 0.5 * diff(cell_boundaries)
@info h0.(cell_centers)
@info cell_centers
u, h = leap_frog_swe_staggered(dx, dt, time_steps, grid_points, h0, 0.1)

[ Info: [0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[ Info: [50000.0, 150000.0, 250000.0, 350000.0, 450000.0, 550000.0, 650000.0, 750000.0, 850000.0, 950000.0, 1.05e6, 1.15e6, 1.25e6, 1.35e6, 1.45e6, 1.55e6, 1.65e6, 1.75e6, 1.85e6, 1.95e6, 2.05e6, 2.15e6, 2.25e6, 2.35e6, 2.45e6, 2.55e6, 2.65e6, 2.75e6, 2.85e6, 2.95e6, 3.05e6, 3.15e6, 3.25e6, 3.35e6, 3.45e6, 3.55e6, 3.65e6, 3.75e6, 3.85e6, 3.95e6, 4.05e6, 4.15e6, 4.25e6, 4.35e6, 4.45e6, 4.55e6, 4.65e6, 4.75e6, 4.85e6, 4.95e6, 5.05e6, 5.15e6, 5.25e6, 5.35e6, 5.45e6, 5.55e6, 5.65e6, 5.75e6, 5.85e6, 5.95e6, 6.05e6, 6.15e6, 6.25e6, 6.35e6, 6.45e6, 6.55e6, 6.65e6, 6.75e6, 6.85e6, 6.95e6, 7.05e6, 7.15e6, 7.25e6, 7.35e6, 7.45e6, 7.55e6, 7.65e6, 7.75e6, 7.85e6, 7.95e6, 8.05e6, 8.15e6, 8.25e6,

([0.0 0.0 … 0.0018026001764237568 0.0017496313222034056; 0.0 0.0 … -0.0017089121730788064 -0.0016895774865082816; … ; 0.0 0.0 … -0.00021203412995985684 -0.0003302624626167104; 0.0 0.0 … -0.0012494063068985227 -0.0011565527740902367], [0.0 0.0 … -0.016157828701474742 -0.017550207903883005; 0.0 0.0 … 0.0023858159681754295 0.0035905304668400013; … ; 0.0 0.0 … -0.05724889733379208 -0.0576264272003774; 0.0 0.0 … 0.036659867080717626 0.037855249809244006])

In [8]:
import TOML
using Dates

include("shallowwater.jl")

function simulate_swe1d(;config_path::String, img_path::String)
    # declare constants
    h0(x)= (450.0 <= x / 1000.0 <= 550.0) ? 1 : 0   # initial function for the height
    STANDARD_GRAVITY = 9.81                         # standard gravity (m/s^2)
    FLUID_DEPTH = 4e3                               # mean fluid depth (m)
    
    filename_gif_swe1d = "swe_1d.gif"
    filename_gif_swe2d = "swe_2d.gif"
    filename_staggered = "swe1d_staggered_grid"
    filename_unstaggered = "swe1d_unstaggered_grid"
    sampling_freq = 20
    fps = 15

    # load config
    config = TOML.parsefile(config_path)
    config_swe1d = config["swe1d"]
    
    dt = config_swe1d["dt"]
    dx = config_swe1d["dx"]
    gamma = config_swe1d["gamma"]
    grid_points = config_swe1d["grid_points"]
    time_steps = config_swe1d["time_steps"]
    indices = config_swe1d["indices_plotting"]
    
    # force consistency for indices for plotting
    indices = [i for i in indices if i <= time_steps && i >= 1]
    
    cell_boundaries = dx * collect(0:grid_points)
    cell_centers = cell_boundaries[1:end-1] + 0.5 * diff(cell_boundaries)
    
    @info "$(now()) - Start computation of SWE in 1D"
    # index a for the A grid (unstaggered) and c for the C grid (staggered)
    u_a, h_a = leap_frog_swe(dx, dt, time_steps, grid_points, h0, GAMMA,:periodic, STANDARD_GRAVITY, FLUID_DEPTH)
    @info "$(now()) - Computation of SWE in 1D with unstaggered grid done"
    u_c, h_c = leap_frog_swe_staggered(dx, dt, time_steps, grid_points, h0, GAMMA, :periodic, STANDARD_GRAVITY, FLUID_DEPTH)
    @info "$(now()) - Computation of SWE in 1D with staggered grid done"
    
    
    @info typeof(indices)
    @info indices
    #create_gif_1dswe(cell_centers::Vector{T}, y::Matrix{T},  filename::String, img_path::String, time_steps::Integer, m::Integer, fps::Integer)
    create_gif_1dswe(cell_centers, h_c,
        filename = filename_gif_swe1d,
        img_path = img_path,
        time_steps = time_steps,
        m = sampling_freq,
        fps = fps)
    
    create_plots_1dswe(cell_centers, u_a, indices,
        filename = "velocity_$(filename_unstaggered)",
        img_path = img_path,
        quantity = "Velocity",
        ylabel = "u(x,t)")
    
    create_plots_1dswe(cell_centers, h_a, indices,
        filename = "height_$(filename_unstaggered)",
        img_path = img_path,
        quantity = "Height",
        ylabel = "h(x,t)")
    
    create_plots_1dswe(cell_centers, u_c, indices,
        filename = "velocity_$(filename_staggered)",
        img_path = img_path,
        quantity = "Velocity",
        ylabel = "u(x,t)")
    
    create_plots_1dswe(cell_centers, h_c, indices,
        filename = "height_$(filename_staggered)",
        img_path = img_path,
        quantity = "Height",
        ylabel = "h(x,t)")
    
    @info "$(now()) - Saved plots for 1D SWE"

end

#simulate_swe1d("./config_swe.toml")



simulate_swe1d (generic function with 1 method)

In [12]:
import TOML
using Dates

include("shallowwater.jl")

function simulate_swe2d(;config_path::String, img_path::String)
    
    # declare constants
    h0(x,y) = (45 * dx <= x <= 55 * dx &&  45 * dy <= y <= 55 * dy ) ? 1 : 0   # initial function for the height   
    filename_gif_swe1d = "swe_1d.gif"
    filename_gif_swe2d = "swe_2d.gif"
    filename_staggered = "swe1d_staggered_grid"
    filename_unstaggered = "swe1d_unstaggered_grid"
    filename_no_coriolis = "swe2d_no_coriolis"
    filename_coriolis = "swe2d_coriolis"
    sampling_freq = 20
    fps = 15
    
    # load configuration
    config = TOML.parsefile(config_path)
    config_swe2d = config["swe2d"]
    
    dt = config_swe2d["dt"]
    dx = config_swe2d["dx"]
    dy = config_swe2d["dy"]
    Nx = config_swe2d["Nx"]
    Ny = config_swe2d["Ny"]
    gamma = config_swe2d["gamma"]
    time_steps = config_swe2d["time_steps"]
    indices = config_swe2d["indices_plotting"]
    coriolis_1 = config_swe2d["coriolis_1"]  # no coriolis
    coriolis_2 = config_swe2d["coriolis_2"]  # coriolis f = 1e-4
    
    # force consistency for indices for plotting
    indices = [i for i in indices if i <= time_steps && i >= 1]
    
    # computation
    @info "$(now()) - Start Simulation of Shallow Water Equation in 2D"
    u_no_coriolis, v_no_coriolis, h_no_coriolis = swe_arakawa(Nx, Ny, dx, dy, dt, time_steps, h0, coriolis_1, gamma)
    @info "$(now()) - Computation of SWE with Coriolis with f = $(coriolis_1) done"
    u_coriolis, v_coriolis, h_coriolis = swe_arakawa(Nx, Ny, dx, dy,  dt, time_steps, h0, coriolis_2, gamma)
    @info "$(now()) - Computation of SWE with Coriolis with f = $(coriolis_2) done"
    
    @info "$(now()) - Start creating Plots"
    create_quiver_gif(u_coriolis, v_coriolis, h_coriolis;
                       dx=dx, dy=dx,
                       filename="swe2d_coriolis.gif",
                       img_path=img_path)
    
    plot_height_heatmap(h_coriolis, indices, filename_coriolis, img_path, dx, dy)
    plot_height_heatmap(h_no_coriolis, indices, filename_no_coriolis, img_path, dx, dy)
    @info "$(now()) - Plots for the 2D SWE created and these can be found in $(img_path)"
end
simulate_swe1d(config_path="./config_swe.toml", img_path="./")
simulate_swe2d(config_path="./config_swe.toml", img_path="./")

[ Info: 2025-08-14T09:39:03.228 - Start computation of SWE in 1D
[ Info: 2025-08-14T09:39:03.729 - Computation of SWE in 1D with unstaggered grid done
[ Info: 2025-08-14T09:39:04.151 - Computation of SWE in 1D with staggered grid done
[ Info: Vector{Int64}
[ Info: [1, 60, 120]
[ Info: Saved animation to /home/niclas/uni_leipzig/07_Veranstaltungen_SoSe25/Meterologie/PraktikumsBericht/src/swe_1d.gif
[ Info: 2025-08-14T09:39:06.137 - Saved animation of 1D Shallow Water Equation to swe_1d.gif
[ Info: 2025-08-14T09:39:06.403 - Saved plots for 1D SWE
[ Info: 2025-08-14T09:39:06.770 - Start Simulation of Shallow Water Equation in 2D
[ Info: 2025-08-14T09:39:09.476 - Computation of SWE with Coriolis with f = 0.0 done
[ Info: 2025-08-14T09:39:10.933 - Computation of SWE with Coriolis with f = 0.0001 done
[ Info: 2025-08-14T09:39:10.933 - Start creating Plots
[ Info: Saved animation to /home/niclas/uni_leipzig/07_Veranstaltungen_SoSe25/Meterologie/PraktikumsBericht/src/swe2d_coriolis.gif
[ Info: